<a href="https://colab.research.google.com/github/pattalkslaw-del/working-thoughts/blob/main/ComfyUI%20Colab%20Upgraded.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⚡ ComfyUI: Professional Edition

### 🔴 **Brought to you by [AI With Chucky](https://youtube.com/@AIWithChucky)**

### 🖥️ **High-Performance AI Environment**
This notebook provides a stable, persistent, and optimized environment for running ComfyUI on Google Colab.

**Key Features:**
- **Hybrid Storage:** Executes code on the local VM for speed, but persists data (Models, Output, Nodes) to Google Drive.
- **Modern Downloader:** Python-native, high-speed downloader with visual progress bars.
- **Memory Safety:** Prevents OOM (Out of Memory) errors via selectable GPU profiles.

In [ ]:
#@title 1. System Initialization
#@markdown **Run this cell first.** <br>
#@markdown This mounts Drive and sets up your environment.

import os
import shutil
import subprocess
from google.colab import drive

# --- Configuration ---
MOUNT_DRIVE = True
UPDATE_COMFY_UI = True #@param {type:"boolean"}
INSTALL_COMFYUI_MANAGER = True #@param {type:"boolean"}

LOCAL_WORKSPACE = "/content/ComfyUI"
DRIVE_WORKSPACE = "/content/drive/MyDrive/ComfyUI"

# 1. Mount Google Drive
if MOUNT_DRIVE:
    print("💾 Mounting Google Drive...")
    drive.mount('/content/drive')

# 2. Setup ComfyUI Core
if not os.path.exists(LOCAL_WORKSPACE):
    print("📦 Cloning ComfyUI repository...")
    !git clone https://github.com/comfyanonymous/ComfyUI {LOCAL_WORKSPACE}
else:
    if UPDATE_COMFY_UI:
        print("🔄 Checking for ComfyUI updates...")
        !cd {LOCAL_WORKSPACE} && git pull

# 3. Configure Storage Symlinks
# Redirects heavy folders to Drive for persistence, keeps code local for speed.
directories_to_sync = [
    "models",
    "custom_nodes",
    "output",
    "input",
    "user"
]

print("🔗 Configuring persistent storage...")
for d in directories_to_sync:
    local_path = os.path.join(LOCAL_WORKSPACE, d)
    drive_path = os.path.join(DRIVE_WORKSPACE, d)

    # Ensure directory exists on Drive
    if not os.path.exists(drive_path):
        os.makedirs(drive_path, exist_ok=True)

    # Remove local directory to replace with symlink
    if os.path.exists(local_path) and not os.path.islink(local_path):
        shutil.rmtree(local_path)

    # Create Symlink
    if not os.path.exists(local_path):
        os.symlink(drive_path, local_path)

# 4. Install ComfyUI Manager (Optional)
if INSTALL_COMFYUI_MANAGER:
    manager_path = os.path.join(LOCAL_WORKSPACE, "custom_nodes", "ComfyUI-Manager")
    if not os.path.exists(manager_path):
        print("📦 Installing ComfyUI Manager...")
        !git clone https://github.com/ltdrdata/ComfyUI-Manager.git {manager_path}
    else:
        print("✅ ComfyUI Manager is installed.")
        !cd {manager_path} && git pull

# 5. Install Python Dependencies
print("🛠️ Installing Python environment requirements...")
!cd {LOCAL_WORKSPACE} && pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121
!pip install insightface onnxruntime-gpu

print("\n✅ [SYSTEM READY] Initialization complete.")

In [ ]:
#@title 2. Model & Node Downloader
#@markdown Paste download links below. This supports **Checkpoints, Diffusion Models (Flux/UNET), Text Encoders (CLIP/T5), CLIP Vision, VAEs, LoRAs, and ControlNets**.

import os
import requests
from urllib.parse import urlparse, unquote
from tqdm.auto import tqdm

WORKSPACE = "/content/ComfyUI"

# --- Input Resources ---
CHECKPOINT_URLS = "" #@param {type:"string"}
UNET_DIFFUSION_URLS = "" #@param {type:"string"}
TEXT_ENCODER_URLS = "" #@param {type:"string"}
CLIP_VISION_URLS = "" #@param {type:"string"}
VAE_URLS = "" #@param {type:"string"}
LORA_URLS = "" #@param {type:"string"}
CONTROLNET_URLS = "" #@param {type:"string"}
UPSCALE_MODELS_URLS = "" #@param {type:"string"}
EMBEDDING_URLS = "" #@param {type:"string"}
CUSTOM_NODE_URLS = "" #@param {type:"string"}

# --- Downloader Logic ---
DIRS = {
    "checkpoints":    os.path.join(WORKSPACE, "models/checkpoints"),
    "unet":           os.path.join(WORKSPACE, "models/unet"),
    "clip":           os.path.join(WORKSPACE, "models/clip"),
    "clip_vision":    os.path.join(WORKSPACE, "models/clip_vision"),
    "vae":            os.path.join(WORKSPACE, "models/vae"),
    "loras":          os.path.join(WORKSPACE, "models/loras"),
    "controlnet":     os.path.join(WORKSPACE, "models/controlnet"),
    "upscale_models": os.path.join(WORKSPACE, "models/upscale_models"),
    "embeddings":     os.path.join(WORKSPACE, "models/embeddings"),
    "custom_nodes":   os.path.join(WORKSPACE, "custom_nodes")
}

def get_filename(url, response):
    """Smartly determines filename from Content-Disposition or URL."""
    if "Content-Disposition" in response.headers:
        import re
        fname = re.findall('filename="?([^"]+)"?', response.headers["Content-Disposition"])
        if fname: return fname[0]
    return unquote(os.path.basename(urlparse(url).path))

def download_file(url, target_dir):
    try:
        # Stream the download to get headers first
        response = requests.get(url, stream=True, allow_redirects=True)
        response.raise_for_status()

        filename = get_filename(url, response)
        file_path = os.path.join(target_dir, filename)
        total_size = int(response.headers.get('content-length', 0))

        if os.path.exists(file_path):
            print(f"   ⏩ Skipping (Exists): {filename}")
            return

        # Modern Progress Bar Log
        print(f"   📥 Downloading: {filename}")

        # The Progress Bar (Auto-Stretching)
        with tqdm(
            total=total_size,
            unit='B',
            unit_scale=True,
            unit_divisor=1024,
            desc="      🚀 Progress",
            dynamic_ncols=True
        ) as bar:
            with open(file_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=1024*1024): # 1MB chunks
                    if chunk:
                        f.write(chunk)
                        bar.update(len(chunk))

        print("      ✅ Download Complete\n")

    except Exception as e:
        print(f"   ❌ Failed to download: {url}")
        print(f"      Error: {e}\n")

def process_downloads(urls_str, target_dir, is_node=False):
    if not urls_str.strip(): return

    url_list = [u.strip() for u in urls_str.replace(',', '\n').split('\n') if u.strip()]
    if not os.path.exists(target_dir): os.makedirs(target_dir, exist_ok=True)

    print(f"📂 Category: {os.path.basename(target_dir)}")

    for url in url_list:
        if is_node:
            node_name = url.split('/')[-1].replace('.git', '')
            node_path = os.path.join(target_dir, node_name)
            if not os.path.exists(node_path):
                print(f"   ⬇️ Cloning Node: {node_name}...")
                !git clone {url} {node_path}
                # Auto-install requirements
                req = os.path.join(node_path, "requirements.txt")
                if os.path.exists(req):
                    print(f"      📦 Installing requirements...")
                    !pip install -r "{req}"
                print("      ✅ Installed\n")
            else:
                print(f"   ⏩ Node exists: {node_name}\n")
        else:
            download_file(url, target_dir)

# --- Execution ---
process_downloads(CHECKPOINT_URLS,     DIRS["checkpoints"])
process_downloads(UNET_DIFFUSION_URLS, DIRS["unet"])
process_downloads(TEXT_ENCODER_URLS,   DIRS["clip"])
process_downloads(CLIP_VISION_URLS,    DIRS["clip_vision"])
process_downloads(VAE_URLS,            DIRS["vae"])
process_downloads(LORA_URLS,           DIRS["loras"])
process_downloads(CONTROLNET_URLS,     DIRS["controlnet"])
process_downloads(UPSCALE_MODELS_URLS, DIRS["upscale_models"])
process_downloads(EMBEDDING_URLS,      DIRS["embeddings"])
process_downloads(CUSTOM_NODE_URLS,    DIRS["custom_nodes"], is_node=True)

print("🎉 All tasks finished.")

In [ ]:
# @title 3. Session Anti-Disconnect
%%html
<b>🔊 Keep-Alive Audio</b><br>
<i>Running this silent audio loop prevents the browser tab from sleeping.</i><br>
<audio src="https://raw.githubusercontent.com/anars/blank-audio/master/10-minutes-of-silence.mp3" autoplay loop controls style="width: 300px;" />

In [ ]:
#@title 4. Start ComfyUI Session
#@markdown Select your preferred memory profile and launch the application.

import subprocess
import threading
import time
import socket
import os

# --- Session Configuration ---
#@markdown **Performance Profile:**
MEMORY_PROFILE = "Standard (Auto-Detect)" #@param ["Standard (Auto-Detect)", "Low VRAM (T4 GPU / Heavy Models)", "High VRAM (A100 GPU Only)"]
#@markdown **Visual Settings:**
LIVE_GENERATION_PREVIEWS = True #@param {type:"boolean"}

# --- Argument Logic ---
ARGS = ""

if "Low VRAM" in MEMORY_PROFILE:
    ARGS += " --lowvram"
elif "High VRAM" in MEMORY_PROFILE:
    ARGS += " --highvram"

if LIVE_GENERATION_PREVIEWS:
    ARGS += " --preview-method auto"

WORKSPACE = "/content/ComfyUI"

# 1. Initialize Network Tunnel
print("🌐 Initializing Cloudflare secure tunnel...")
!wget -q -nc https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

def start_tunnel(port):
    while True:
        time.sleep(1)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()

    print("\n🟢 ComfyUI is Online. Generating Access Link...\n")
    p = subprocess.Popen(["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com" in l:
            url = l[l.find("http"):].strip()
            print("\n" + "="*60)
            print(f"🔗 ACCESS LINK: {url}")
            print("="*60 + "\n")

threading.Thread(target=start_tunnel, daemon=True, args=(8188,)).start()

# 2. Launch Application
if os.path.exists(os.path.join(WORKSPACE, "main.py")):
    %cd {WORKSPACE}
    print(f"🚀 Launching ComfyUI with profile: [{MEMORY_PROFILE}]...")
    !python main.py --dont-print-server --listen 127.0.0.1 --port 8188 {ARGS}
else:
    print(f"❌ [ERROR] System files missing. Please re-run Cell 1.")